# SPARQL Workshop - Section 08: Participant Exercises

Here are the solutions to the exercises in Section 07.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>


### 08.01 - Simple look-up by property

Display all the properties and values of the artist with name Kate Beckinsale, ordered by predicate.

PREFIXes have been included for you to help you with this effort.


In [ ]:
%%sparql --store-to results_08_01

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>

SELECT * WHERE {
  ?artist rdf:type class:Artist .
  ?artist dataProp:name "Kate Beckinsale" .
  ?artist ?property ?value
} ORDER BY ?property

In [ ]:
!pip install ipython_unittest
%load_ext ipython_unittest

In [ ]:
%%unittest

assert results_08_01['results']['bindings'] == [{'artist': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/resource/nm0000295'}, 
                          'property': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/birthYear'}, 
                          'value': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '1973'}},
                         {'artist': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/resource/nm0000295'}, 
                          'property': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/name'}, 
                          'value': {'type': 'literal', 'value': 'Kate Beckinsale'}},
                         {'artist': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/resource/nm0000295'}, 
                          'property': {'type': 'uri', 'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'}, 
                          'value': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/class/Artist'}}, 
                         {'artist': {'type': 'uri', 'value': 'http://aws.amazon.com/neptune/csv2rdf/resource/nm0000295'}, 
                          'property': {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}, 
                          'value': {'type': 'literal', 'value': 'Kate Beckinsale'}}]

### 08.02 - Single hop traversal

Find out all the movies which Denzel Washington acted in. (Order the movies by id before limiting to 10.)



In [ ]:
%%sparql --store-to results_08_02

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT ?title WHERE {
  ?artist rdf:type class:Artist .
  ?artist dataProp:name "Denzel Washington" .
  ?movie objProp:actor ?artist .
  ?movie dataProp:title ?title
} ORDER BY (?movie) LIMIT 10

In [ ]:
%%unittest

assert results_08_02['results']['bindings'] == [{'title': {'type': 'literal', 'value': 'Cry Freedom'}},
                         {'title': {'type': 'literal', 'value': 'For Queen & Country'}},
                         {'title': {'type': 'literal', 'value': 'Glory'}},
                         {'title': {'type': 'literal', 'value': 'The Mighty Quinn'}},
                         {'title': {'type': 'literal', 'value': 'Heart Condition'}},
                         {'title': {'type': 'literal', 'value': "Mo' Better Blues"}},
                         {'title': {'type': 'literal', 'value': 'Mississippi Masala'}},
                         {'title': {'type': 'literal', 'value': 'Ricochet'}},
                         {'title': {'type': 'literal', 'value': 'Malcolm X'}},
                         {'title': {'type': 'literal', 'value': 'The Pelican Brief'}}]

### 08.03 - Two hop traversal

Find 5 unique actors or actresses which were paired with Julia Roberts. (Use `DISTINCT` to deduplicate the actors, and then order by id before limiting to 5.)


In [ ]:
%%sparql --store-to results_08_03

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT DISTINCT ?name WHERE {
  ?artist rdf:type class:Artist .
  ?artist dataProp:name "Julia Roberts" .
  ?movie objProp:actress ?artist .
  ?movie objProp:actor|objProp:actress ?costars .
  ?costars dataProp:name ?name
} ORDER BY (?costars) LIMIT 5

In [ ]:
%%unittest

assert results_08_03['results']['bindings'] == [{'name': {'type': 'literal', 'value': 'Sophia Loren'}},
                         {'name': {'type': 'literal', 'value': 'Marcello Mastroianni'}},
                         {'name': {'type': 'literal', 'value': 'Brad Pitt'}},
                         {'name': {'type': 'literal', 'value': 'Woody Allen'}},
                         {'name': {'type': 'literal', 'value': 'Jennifer Aniston'}}]

### 08.04 - Using FILTER Expressions

Find the Person who lives in Belfast and follows Gillian Anderson.

In [ ]:
%%sparql --store-to results_08_04

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT ?firstname ?lastname WHERE {
  ?artist rdf:type class:Artist .
  ?artist dataProp:name "Gillian Anderson" .
  ?person objProp:follows ?artist .
  ?person dataProp:firstName ?firstname .
  ?person dataProp:lastName ?lastname .
  ?person objProp:isLocatedIn ?place .
  ?place dataProp:name "Belfast" .
}

In [ ]:
%%unittest

assert results_08_04['results']['bindings'] == [{'firstname': {'type': 'literal', 'value': 'David'},
                          'lastname': {'type': 'literal', 'value': 'Bell'}}]

### 08.05 - Using FILTER on edges & range queries

List persons who have given The Prestige a rating of less than 3, ordered by first name.

In [ ]:
%%sparql --store-to results_08_05

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT ?firstname ?lastname WHERE {
  ?movie rdf:type class:Movie .
  ?movie dataProp:title "The Prestige" .
  GRAPH ?redge {
      ?person objProp:rated ?movie
  }
  ?redge dataProp:rating ?rating .
  FILTER (?rating < 3)
  ?person dataProp:firstName ?firstname .
  ?person dataProp:lastName ?lastname .
} ORDER BY (?firstname)

In [ ]:
%%unittest

assert results_08_05['results']['bindings'] == [{'firstname': {'type': 'literal', 'value': 'Amber'},
                          'lastname': {'type': 'literal', 'value': 'Azami'}},
                         {'firstname': {'type': 'literal', 'value': 'Jharana'},
                          'lastname': {'type': 'literal', 'value': 'Gurung'}},
                         {'firstname': {'type': 'literal', 'value': 'Paulo'},
                          'lastname': {'type': 'literal', 'value': 'Silva'}}]

### 08.06 - Aggregating and Counting Results

Find all Artists that worked with John Cleese and the number of times they appeared with him. Order the results by number of appearances and return the first five. (Hint: Use `GROUP BY and COUNT` )

In [ ]:
%%sparql --store-to results_08_06

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>
PREFIX objProp: <http://aws.amazon.com/neptune/csv2rdf/objectProperty/>

SELECT ?costarnames (COUNT(?costarnames) as ?appearances) WHERE {
    ?artist rdf:type class:Artist .
    ?artist dataProp:name "John Cleese" .
    ?movie objProp:actor ?artist .
    ?movie objProp:actor|objProp:actress ?costars .
    ?costars dataProp:name ?costarnames
    FILTER (?costars != ?artist)
} GROUP BY (?costarnames) ORDER BY DESC(?appearances) LIMIT 5

In [ ]:
%%unittest

assert results_08_06['results']['bindings'] == [{'appearances': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                  'type': 'literal',
                  'value': '6'},
                  'costarnames': {'type': 'literal', 'value': 'Michael Palin'}},
                 {'appearances': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                  'type': 'literal',
                                  'value': '5'},
                  'costarnames': {'type': 'literal', 'value': 'Terry Gilliam'}},
                 {'appearances': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                  'type': 'literal',
                                  'value': '5'},
                  'costarnames': {'type': 'literal', 'value': 'Graham Chapman'}},
                 {'appearances': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                  'type': 'literal',
                                  'value': '4'},
                  'costarnames': {'type': 'literal', 'value': 'Eric Idle'}},
                 {'appearances': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
                                  'type': 'literal',
                                  'value': '3'},
                  'costarnames': {'type': 'literal', 'value': 'Terry Jones'}}]